# Setup

In [ ]:
!pip install -q condacolab gdown

In [ ]:
import condacolab
condacolab.install()

In [ ]:
# !rm -rf /content/ghost-2-0-FaceSwap_n_HeadSwap

In [ ]:
import os
import zipfile

%cd /content

! git clone --recurse-submodules https://github.com/manishh-07/ghost-2-0-FaceSwap_n_HeadSwap.git
%cd ghost-2-0-FaceSwap_n_HeadSwap
! git submodule init
! git submodule update
os.makedirs("aligner_checkpoints", exist_ok=True)
os.makedirs("blender_checkpoints", exist_ok=True)
os.makedirs("src/losses/gaze_models", exist_ok=True)
os.makedirs("weights", exist_ok=True)

! wget -O aligner_checkpoints/aligner_1020_gaze_final.ckpt https://github.com/ai-forever/ghost-2.0/releases/download/aligner/aligner_1020_gaze_final.ckpt
! wget -O blender_checkpoints/blender_lama.ckpt https://github.com/ai-forever/ghost-2.0/releases/download/aligner/blender_lama.ckpt
! wget -O weights/backbone50_1.pth https://github.com/ai-forever/ghost-2.0/releases/download/aligner/backbone50_1.pth
! wget -O weights/vgg19-d01eb7cb.pth https://github.com/ai-forever/ghost-2.0/releases/download/aligner/vgg19-d01eb7cb.pth
! wget -O weights/segformer_B5_ce.onnx https://github.com/ai-forever/ghost-2.0/releases/download/aligner/segformer_B5_ce.onnx
! wget -O gaze_models.zip https://github.com/ai-forever/ghost-2.0/releases/download/aligner/gaze_models.zip

with zipfile.ZipFile("gaze_models.zip", 'r') as zip_ref:
    zip_ref.extractall("src/losses/")

os.remove("gaze_models.zip")
print("✅ All models downloaded and extracted successfully.")

! mkdir repos

%cd repos

! git clone --recurse-submodules https://github.com/yfeng95/DECA
%cd DECA
! git submodule init
! git submodule update
%cd ..

! git clone --recurse-submodules https://github.com/jagrut-thakare/emoca
%cd emoca

! git submodule init
! git submodule update

os.makedirs("gdl_apps/EmotionRecognition", exist_ok=True)
os.makedirs("assets/EmotionRecognition/image_based_networks", exist_ok=True)

!wget -O ResNet50.zip https://github.com/anastasia-yaschenko/emoca/releases/download/resnet/ResNet50.zip

with zipfile.ZipFile("ResNet50.zip", 'r') as zip_ref:
    zip_ref.extractall("gdl_apps/EmotionRecognition")
    zip_ref.extractall("assets/EmotionRecognition/image_based_networks")

os.remove("ResNet50.zip")

print("✅ ResNet checkpoints downloaded, unpacked, and ZIP removed.")
%cd ..

! git clone --recurse-submodules https://github.com/anastasia-yaschenko/BlazeFace_PyTorch

%cd BlazeFace_PyTorch
! git submodule init
! git submodule update
%cd ..

! git clone --recurse-submodules https://github.com/chroneus/stylematte

%cd stylematte

! git submodule init
! git submodule update

os.makedirs("stylematte/checkpoints", exist_ok=True)
os.makedirs("stylegan3", exist_ok=True)

!wget -O stylematte/checkpoints/stylematte_pure.pth https://github.com/chroneus/stylematte/releases/download/weights/stylematte_pure.pth
!wget -O stylematte/checkpoints/stylematte_synth.pth https://github.com/chroneus/stylematte/releases/download/weights/stylematte_synth.pth
!wget -O stylegan3/animals.pkl https://github.com/chroneus/stylematte/releases/download/weights/animals.pkl
!wget -O stylegan3/humans.pkl https://github.com/chroneus/stylematte/releases/download/weights/humans.pkl
%cd ..

%cd ..

In [ ]:
%cd /content/ghost-2-0-FaceSwap_n_HeadSwap

In [ ]:
! conda config --add channels conda-forge
! conda config --set channel_priority strict
! conda create -n ghost python=3.10

In [ ]:
! conda run -n ghost pip install -r requirements.txt

In [ ]:
! conda install -n ghost -c pytorch3d-nightly pytorch3d
! conda run -n ghost pip install face-alignment==1.3.5 facenet-pytorch==2.5.2 -U git+https://github.com/facebookresearch/fvcore

In [ ]:
!conda run -n ghost conda install scikit-image

In [ ]:
!conda run -n ghost conda install -c conda-forge imgaug

In [ ]:
!conda run -n ghost conda install numpy==1.23.5

In [ ]:
!conda run -n ghost conda install tensorboardX tensorboard

# DECA Model

In [ ]:
import gdown

file_id = "1rp8kdyLPvErw2dTmqtjISRVvQLj6Yzje"
destination = "/content/ghost-2-0-FaceSwap_n_HeadSwap/repos/DECA/data/deca_model.tar"

gdown.download(id=file_id, output=destination, quiet=False)

In [ ]:
import os
from google.colab import userdata

# Create data directory
os.makedirs("data", exist_ok=True)

# Set FLAME credentials (assumed to be already set)
username = userdata.get('DECA')
password = userdata.get('DECA_Password')

# Check if credentials are provided
if not username or not password:
    raise ValueError("Environment variables  must be set.")

# Encode username and password for URL
import urllib.parse
username_enc = urllib.parse.quote_plus(username)
password_enc = urllib.parse.quote_plus(password)

# Create the download command
download_url = f"https://download.is.tue.mpg.de/download.php?domain=flame&sfile=FLAME2020.zip&resume=1"
post_data = f"username={username_enc}&password={password_enc}"

# Use wget with POST data
!wget --post-data="{post_data}" "{download_url}" -O DECA.zip --no-check-certificate --continue

# Unzip and move model file
!unzip DECA.zip -d ./temp
!mv ./temp/* /content/ghost-2-0-FaceSwap_n_HeadSwap/repos/DECA/data/
!rm -rf ./temp/
!rm -rf DECA.zip

# EMOCA ResNet50

In [ ]:
import os
import gdown

# Create the directory
os.makedirs("/ps/scratch/rdanecek/FaceRecognition/", exist_ok=True)
# Google Drive file ID
file_id = "1gSUm_sZFOjcHij4wBptI3CEvmWQzVHoS"
output_path = "/ps/scratch/rdanecek/FaceRecognition/resnet50_ft_weight.pkl"

# Remove the file if it exists before downloading
if os.path.exists(output_path):
    os.remove(output_path)
    print(f"Removed existing file: {output_path}")

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
print(f"Downloaded file to: {output_path}")

# Training

In [ ]:
%cd /content/ghost-2-0-FaceSwap_n_HeadSwap

In [ ]:
import os
os.environ["MPLBACKEND"] = "agg"

In [ ]:
!conda run -n ghost pip install lightning==2.5.0.post0
!conda run -n ghost pip install omegaconf==2.3.0
!conda run -n ghost pip install transformers==4.48.2
!conda run -n ghost pip install adabound==0.0.5
!conda run -n ghost pip install torchfile==0.1.0
!conda run -n ghost pip install scikit-image==0.25.1
!conda run -n ghost pip install imgaug==0.4.0
!conda run -n ghost pip install gdown
!conda run -n ghost pip install matplotlib==3.10.0
!conda run -n ghost pip install h5py==3.12.1
!conda run -n ghost pip install facenet-pytorch
!conda run -n ghost pip install face-alignment
!conda run -n ghost pip install mediapipe==0.10.21
!conda run -n ghost pip install wandb==0.19.6
!conda run -n ghost pip install kornia==0.8.0
!conda run -n ghost pip install lpips==0.1.4
!conda run -n ghost pip install pytorch-msssim==1.0.0
!conda run -n ghost pip install pytorch_fid
!conda run -n ghost pip install einops==0.8.0
# !conda run -n ghost pip install "numpy>=1.23.5,<2.5.0"
! conda run -n ghost pip install chumpy==0.70
! conda run -n ghost pip install Pillow

! conda run -n ghost pip install -r requirements.txt

In [ ]:
import os
import cv2
from PIL import Image
import shutil
 
def repair_and_save_as_png(src_dir, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)
 
    for person in os.listdir(src_dir):
        person_path = os.path.join(src_dir, person)
        dst_person_path = os.path.join(dst_dir, person)
        os.makedirs(dst_person_path, exist_ok=True)
 
        for img_file in os.listdir(person_path):
            src_img_path = os.path.join(person_path, img_file)
            file_base, _ = os.path.splitext(img_file)
            dst_img_path = os.path.join(dst_person_path, f"{file_base}.png")
 
            try:
                # Attempt OpenCV read
                img = cv2.imread(src_img_path)
                if img is None:
                    print(f"[SKIP] cv2 couldn't read: {src_img_path}")
                    continue
 
                # Convert to RGB
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
                # Save as PNG using PIL (ensures compatibility with ML pipelines)
                Image.fromarray(img_rgb).save(dst_img_path)
                print(f"[REPAIRED] Saved: {dst_img_path}")
 
            except Exception as e:
                print(f"[FAIL] {src_img_path} - {e}")
                continue
 
repair_and_save_as_png(
    src_dir='/content/ghost-2.0/src/data/train/',
    dst_dir='/content/ghost-2.0/src/data/ABD_train'
)

In [ ]:
!pip install torch torchvision


In [ ]:

! conda run -n ghost python /content/ghost-2-0-FaceSwap_n_HeadSwap/v2h5.py --input /content/ghost-2-0-FaceSwap_n_HeadSwap/src/data/train --output /content/ghost-2-0-FaceSwap_n_HeadSwap/src/data/train_h5
! conda run -n ghost python /content/ghost-2-0-FaceSwap_n_HeadSwap/v2h5.py --input /content/ghost-2-0-FaceSwap_n_HeadSwap/src/data/test --output /content/ghost-2-0-FaceSwap_n_HeadSwap/src/data/test_h5

In [ ]:
# ! conda run -n ghost python v2h5.py

In [ ]:
!conda run -n ghost pip install numpy==1.23.5

In [ ]:
# ! conda run -n ghost pip install h5py

In [ ]:
!conda run -n ghost pip install --force-reinstall packaging

In [ ]:
! conda run -n ghost python train_aligner.py

In [ ]:
!conda run -n ghost python train_blender.py --config /content/ghost-2-0-FaceSwap_n_HeadSwap/configs/blender.yaml

# Inference

In [ ]:
%cd /content/ghost-2-0-FaceSwap_n_HeadSwap

In [ ]:
import os
os.environ["MPLBACKEND"] = "agg"

In [ ]:
!conda run -n ghost python inference.py --source ./examples/images/elon.jpg --target ./examples/images/jolie.jpg --save_path result1.png

#or if trained model

# !conda run -n ghost python inference.py \
#   --source path/to/source.jpg \
#   --target path/to/target.jpg \
#   --ckpt_a path/to/your_trained_aligner_checkpoint.ckpt \
#   --ckpt_b blender_checkpoints/blender_lama.ckpt \
#   --swap_mode both \
#   --save_path result.png

# **FaceSwap layer using InsightFace Inswapper Model**

In [ ]:
! pip install insightface onnxruntime opencv-python


In [ ]:
from insightface.app import FaceAnalysis
from insightface.model_zoo import get_model
import cv2

src_img = cv2.imread("/content/ghost-2.0/examples/images/elon.jpg") # Source face
ghost_img = cv2.imread("/content/ghost-2.0/result2.png")    # Ghost head-swapped result

app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0)

# Load faces
faces_src = app.get(src_img)
faces_dst = app.get(ghost_img)

# Load face swap model
swapper = get_model('inswapper_128.onnx', download=True)


if faces_src and faces_dst:
    swapped_img = swapper.get(ghost_img, faces_dst[0], faces_src[0], paste_back=True)
    cv2.imwrite("final_output2.jpg", swapped_img)
    print("FaceSwap complete. Output: final_output.jpg")
else:
    print("Face not detected in one or both images.")
